<a href="https://colab.research.google.com/github/Dushanthimadhushika3/FP-Growth-Algorithm/blob/main/FP_Growth_Algorithm(Association).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 212.4MB 62kB/s 
     |████████████████████████████████| 204kB 21.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=7404de37e3494a2c880b931301ae3333dea7b3b329c18686596334e5669bae60
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [2]:
import pandas as pd
import glob

from pyspark.sql import SparkSession

# Import the FPGrowth API from PySpark
from pyspark.ml.fpm import FPGrowth
from pyspark.sql import SQLContext

spark = SparkSession.builder.master("local").appName("Colab")\
        .config('spark.ui.port', '4050').getOrCreate()

In [3]:
csv = glob.glob('/content/drive/MyDrive/L4S1/Big Data/Data/Datasets'+'/*.csv')

df = pd.DataFrame()
# Read the data from the dataset
for f in csv:
  file = pd.read_csv(f)
  df = df.append(file)

In [4]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-01-01 00:00:00 UTC,view,5809910,1602943681873052386,NaN,grattol,5.24,595414620,4adb70bb-edbd-4981-b60f-a05bfd32683a
1,2020-01-01 00:00:09 UTC,view,5812943,1487580012121948301,NaN,kinetics,3.97,595414640,c8c5205d-be43-4f1d-aa56-4828b8151c8a
2,2020-01-01 00:00:19 UTC,view,5798924,1783999068867920626,NaN,zinger,3.97,595412617,46a5010f-bd69-4fbe-a00d-bb17aa7b46f3
3,2020-01-01 00:00:24 UTC,view,5793052,1487580005754995573,NaN,NaN,4.92,420652863,546f6af3-a517-4752-a98b-80c4c5860711
4,2020-01-01 00:00:25 UTC,view,5899926,2115334439910245200,NaN,NaN,3.92,484071203,cff70ddf-529e-4b0c-a4fc-f43a749c0acb


In [5]:
sc = df[['user_session','brand','user_id','event_type']]
sc = sc.loc[sc['event_type']=='purchase']
sc = sc[['user_session','brand']]
sc = sc.dropna()
print(sc.shape)
sc.head()

sc.to_csv('/content/purchase.csv')

(737314, 2)


In [6]:
session_id_col_name = 'user_session'
product_id_col_name = 'brand'
session_id_col_index = -1
product_id_col_index = -1
data = dict()

with open('/content/purchase.csv') as f:
    lis = [line.rstrip().split(',') for line in f]        
    for i, x in enumerate(lis):              
        if i == 0:
            if type(x) is list and session_id_col_name in x and product_id_col_name in x: # column names
                session_id_col_index = x.index(session_id_col_name)
                product_id_col_index = x.index(product_id_col_name)
            else:
                print('Required columns does not exists in given csv file')
                break
        else:
            if x[session_id_col_index] in data:
                data[x[session_id_col_index]] = data[x[session_id_col_index]] + ',' + x[product_id_col_index]
            else:
                data[x[session_id_col_index]] = x[product_id_col_index]


# Create the pandas DataFrame
dataframe = pd.DataFrame(list(data.items()),columns=[session_id_col_name,product_id_col_name])
dataframe.head()
print(dataframe.shape)


(137908, 2)


In [7]:
# Preprocess the data
records = []
for i in range(0,137908):
  record = []
  for j in range(1,2):
    val = dataframe.values[i, j].split(",")
    for k in val:
      if k not in record:
        record.append(k)
  records.append([i, record])

print(record)
# Create the pandas DataFrame
out = pd.DataFrame(records, columns = ['id', 'items'])
out.head()

['ingarden']


,id,items
0,0,[kiss]
1,1,[cnd]
2,2,"[refectocil, cosmoprofi, zinger, runail, irisk]"
3,3,"[roubloff, depilflax, irisk, domix, grattol, m..."
4,4,[concept]


In [8]:
# Convert the pandas dataframe into spark dataframe
sqlContext = SQLContext(spark)
sf = spark.createDataFrame(out,["id", "items"])
sf.show()

+---+--------------------+
| id|               items|
+---+--------------------+
|  0|              [kiss]|
|  1|               [cnd]|
|  2|[refectocil, cosm...|
|  3|[roubloff, depilf...|
|  4|           [concept]|
|  5|             [estel]|
|  6|[runail, irisk, l...|
|  7|            [runail]|
|  8|[ingarden, irisk,...|
|  9|[irisk, runail, b...|
| 10|[runail, concept,...|
| 11|             [irisk]|
| 12|[dartnails, ylloz...|
| 13|[bluesky, grattol...|
| 14|[konad, irisk, it...|
| 15|[grattol, bpw.sty...|
| 16|[irisk, grattol, ...|
| 17|[irisk, runail, n...|
| 18|             [estel]|
| 19|[severina, cosmop...|
+---+--------------------+
only showing top 20 rows



In [9]:
# Create fpGrowth object by instantiating the FPGrowth constructor with the necessary parameters
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.01, minConfidence=0.01)

# Fit the dataframe into the fpGrowth object to get the model prepared
model = fpGrowth.fit(sf)
# Display the frequently occuring items
model.freqItemsets.show()

+--------------------+-----+
|               items| freq|
+--------------------+-----+
|           [staleks]| 4155|
|    [staleks, irisk]| 1552|
|   [staleks, runail]| 1704|
|             [f.o.x]| 2763|
|          [solomeya]| 1798|
|           [bluesky]| 5559|
|    [bluesky, irisk]| 2160|
|[bluesky, irisk, ...| 1445|
|   [bluesky, runail]| 2841|
|             [domix]|10006|
|    [domix, grattol]| 1785|
|   [domix, ingarden]| 1612|
|      [domix, irisk]| 4336|
|[domix, irisk, ru...| 2737|
|     [domix, runail]| 4868|
|           [levrana]| 1580|
|            [uskusi]| 2351|
|           [lianail]| 3591|
|   [lianail, runail]| 1607|
|          [severina]| 7807|
+--------------------+-----+
only showing top 20 rows



In [10]:
# Display the generated association rules
model.associationRules.show()

+-------------------+-----------+-------------------+------------------+--------------------+
|         antecedent| consequent|         confidence|              lift|             support|
+-------------------+-----------+-------------------+------------------+--------------------+
| [severina, runail]|    [irisk]| 0.5723214285714285| 2.102608119010831|0.018592104881515213|
|             [pole]|    [irisk]| 0.4037460978147763| 1.483292046924188|0.014067349247324303|
|             [pole]|   [runail]| 0.5023933402705515|1.5267868566965164| 0.01750442323868086|
|         [nagaraku]|    [irisk]|  0.498050139275766|1.8297484843956082|0.012965165182585493|
|         [nagaraku]|   [runail]| 0.3896935933147632| 1.184289298284501|0.010144444122168403|
|      [milv, irisk]|   [runail]| 0.6210562414266118|1.8874066009092572|0.013131943034486759|
|           [zinger]|    [irisk]|0.41102542130819764|1.5100349992479867|0.020868985120515125|
|           [zinger]|   [runail]|0.46329620108540415| 1.4079

In [11]:
# Predicts what the customer is most likely to buy with the items bought in the dataset
model.transform(sf).show(20)

+---+--------------------+--------------------+
| id|               items|          prediction|
+---+--------------------+--------------------+
|  0|              [kiss]|                  []|
|  1|               [cnd]|                  []|
|  2|[refectocil, cosm...|[staleks, bluesky...|
|  3|[roubloff, depilf...|[runail, severina...|
|  4|           [concept]|                  []|
|  5|             [estel]|[irisk, runail, k...|
|  6|[runail, irisk, l...|[staleks, bluesky...|
|  7|            [runail]|[staleks, bluesky...|
|  8|[ingarden, irisk,...|[grattol, bpw.sty...|
|  9|[irisk, runail, b...|[staleks, domix, ...|
| 10|[runail, concept,...|[estel, irisk, st...|
| 11|             [irisk]|[staleks, bluesky...|
| 12|[dartnails, ylloz...|[staleks, bluesky...|
| 13|[bluesky, grattol...|[irisk, runail, d...|
| 14|[konad, irisk, it...|[staleks, bluesky...|
| 15|[grattol, bpw.sty...|[runail, irisk, d...|
| 16|[irisk, grattol, ...|[domix, severina,...|
| 17|[irisk, runail, n...|[staleks, blue